## Python decorators

## Just in time compilation


First, we import the tools that we will need: 

In [0]:
from numba import jit, njit
import numpy as np
import math

explain interpreted code, just in time compilation

In [0]:
def std(xs):
  mean = 0
  for x in xs: 
    mean += x
  mean /= len(xs)
  ms = 0
  for x in xs:
     ms += (x-mean)**2
  std = math.sqrt( ms / len(xs) )
  return std

In [0]:
a = np.random.normal(0, 1, 10000000)

In [4]:
std(a)

0.9999284223716342

Explain decorator and nopython better

In [0]:
c_std = njit(std)

In [6]:
c_std(a)

0.9999284223716342

In [7]:
%timeit std(a)

1 loop, best of 3: 4.67 s per loop


In [8]:
%timeit c_std(a)

10 loops, best of 3: 33.3 ms per loop


The compiled function is 100 times faster! 

But obviously we did not have to go into such trouble to compute the standard deviation of our array. For that, we can simply use numpy: 

In [11]:
a.std()

0.9999284223716103

In [12]:
%timeit a.std()

10 loops, best of 3: 52.1 ms per loop


## Calculation of pi

the number pi can be estimated with a very elegant Monte Carlo method. 

Just consider a square of side L=2, centred on (0,0). In this square, we fit a circle of radius R=1, as shown in the figure below. 

The ratio of the circle area to the square area is 

$$r = \frac{A_c}{A_s} = \frac{\pi R^2}{L^2} = \pi / 4$$

so 

$$\pi = 4 r$$

So if we can estimate this ratio, we can estimate pi! 

And to estimate this ratio, we will simply shoot a large number of points in the square, following a uniform probability distribution. The fraction of the points falling in the circle is an estimator of r. 

Obviously, the more points, the more precise this estimator will be, and the more decimals of pi can be computed. 

Let's implement this method: 


In [0]:
import random 

def pi(npoints): 
  n_in_circle = 0 
  for i in range(npoints):
    x = random.random()
    y = random.random()
    if (x**2+y**2 < 1):
      n_in_circle += 1
  return 4*n_in_circle / npoints

In [40]:
npoints = [10, 100, 10000, int(10e6)]
for np in npoints:
  print(pi(np))


3.2
3.12
3.1572
3.1419156


As you can see, even with N=10 million points, the uncertainty is not great. More specifically, the relative uncertainty on pi can be calculated as 

$$\delta = 1/ \sqrt{N}$$

Here is how the uncertainty evolves with the number of points: 

In [39]:
import math
uncertainty = lambda x: 1/math.sqrt(x)
for np in npoints:
  print('npoints', np, 'delta:', uncertainty(np))

npoints 10 delta: 0.31622776601683794
npoints 100 delta: 0.1
npoints 10000 delta: 0.01
npoints 10000000 delta: 0.00031622776601683794


Clearly, we'll need a lot of points. How fast is our code? 

In [42]:
%timeit pi(10000000)

1 loop, best of 3: 3.82 s per loop


3.82 s for 10 million points. This algorithm is O(N), so if we want to use 1 **billion** points, it will take us about 7 minutes . We don't have that much time, so let's use numba! 

In [0]:
@njit
def fast_pi(npoints): 
  n_in_circle = 0 
  for i in range(npoints):
    x = random.random()
    y = random.random()
    if (x**2+y**2 < 1):
      n_in_circle += 1
  return 4*n_in_circle / npoints

In [5]:
fast_pi( int(1e9) )

3.141515092

This took about 10 s, instead of 7 minutes!

## A more involved example: Finding the closest two points

Numpy features an efficient implementation for most array operations. Indeed, you can use numpy to map any function to all elements of an array, or to perform elementwise operations on several arrays. If numpy can do it, just go for it. 

But sometimes, you'll come up with an expensive algorithm that cannot easily be implemented with numpy. For instance, let's consider the following function, which takes an array of 2D points, and looks for the closest two points.

In [0]:
import sys
def closest(points):
  '''Find the two closest points in an array of points in 2D. 
  Returns the two points, and the distance between them'''
  # we will search for the two points with a minimal
  # square distance. 
  # we use the square distance instead of the distance
  # to avoid a square root calculation for each pair of 
  # points
  mindist2 = 999999.
  mdp1, mdp2 = None, None
  for i in range(len(points)):
    p1 = points[i]
    x1, y1 = p1
    for j in range(i+1, len(points)):
      p2 = points[j]
      x2, y2 = p2
      dist2 = (x1-x2)**2 + (y1-y2)**2
      if dist2 < mindist2:
        # squared distance is improved, 
        # keep it, as well as the two 
        # corresponding points
        mindist2 = dist2
        mdp1,mdp2 = p1,p2
  return mdp1, mdp2, math.sqrt(mindist2)

You might be thinking that this algorithm is quite naive, and it's true! I wrote it like this on purpose. 

You can see that there is a double loop in this algorithm. So if we have N points, we would have to test NxN pairs of points. That's why we say that this algorithm has a complexity of order NxN, denoted O(NxN). 

To improve the situation a bit, please note that the distance between point i and point j is the same as the distance between point j and point i! 
So there is no need to check this combination twice. Also, the distance between point i and itself is zero, and should not be tested...That's why I started the inner loop at i+1. So the combinations that are tested are: 

* (0,1), (0,2), ... (0, N)
* (1,2), (1,3), ... (1, N)
* ...

Another thing to note is that I'm doing all I can to limit the amount of computing power needed for each pair. That's why I decided to minimize the square distance instead of the distance itself, which saves us a call to math.sqrt for every pair of points. 

Still, the algorithm remains O(NxN). 

Let's first run this algorithm on a small sample of 10 points, just to check that it works correctly. 

In [29]:
points = np.random.uniform((-1,-1), (1,1), (10,2))
print(points)
closest(points)

[[-0.24869687  0.5767023 ]
 [ 0.73711118  0.38676633]
 [-0.49424816 -0.84443341]
 [ 0.4190678  -0.44513333]
 [ 0.33067554 -0.80315098]
 [-0.06216111 -0.03192764]
 [-0.91377436  0.69830838]
 [ 0.76917069  0.35888663]
 [-0.89219528  0.95967656]
 [-0.20127953 -0.35096548]]


(array([0.73711118, 0.38676633]),
 array([0.76917069, 0.35888663]),
 0.042486348267704956)

Ok, this looks right, the two points indeed appear to be quite close. Let's see how fast is the calculation:

In [30]:
%timeit closest(points)

10000 loops, best of 3: 77.8 µs per loop


Now, let's increase a bit the number of points in the sample. You will see that the calculation will be much slower. 

In [34]:
points = np.random.uniform((-1,-1), (1,1), (2000,2))
closest(points)

(array([-0.18076454,  0.13168443]),
 array([-0.18117409,  0.13203323]),
 0.0005379514898701553)

In [32]:
%timeit closest(points)

1 loop, best of 3: 3.01 s per loop


Since our algorithm is O(NxN), if we go from 10 to 2,000 points, the algorithm will be 200x200 = 40,000 times slower. 

And this is what we have measured: 2.97 / 78e-6 = 38,000 

Now let's try and speed this up with numba's just in time compilation: 

In [27]:
c_closest = njit(closest)
c_closest(points)

SystemError: ignored

It does not work! But no reason to panic. The important message is :

```
TypeError: cannot convert native ?array(float64, 1d, C) to Python object
```

That's a bit cryptic, but one can always google error messages. What this one means is that numba has trouble converting the native

In [0]:
import numba as nb
@nb.jit('Tuple((float64[:], float64[:], float64))(float64[:,:])', 
        nopython=True)
def c_closest(points):
  mindist2 = 999999.
  mdp1, mdp2 = None, None
  for i in range(len(points)):
    p1 = points[i]
    x1, y1 = p1
    for j in range(i+1, len(points)): 
      p2 = points[j]
      x2, y2 = p2
      dist2 = (x1-x2)**2 + (y1-y2)**2
      if dist2 < mindist2: 
        mindist2 = dist2
        mdp1 = p1
        mdp2 = p2
  return mdp1, mdp2, math.sqrt(mindist2)

In [0]:
c_closest(points)

(array([-0.22269406, -0.80339465]),
 array([-0.2234808 , -0.80320374]),
 0.0008095746393564655)

In [0]:
%timeit closest(points)

1 loop, best of 3: 3.85 s per loop


In [0]:
%timeit c_closest(points)

10 loops, best of 3: 31.8 ms per loop


Again, the compiled code is more than 100 times faster! 